In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from PIL import Image

my_dpi = 192

In [2]:
# Load in array of 201 images, create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(columns=['CR2.2', 'RMSE2.2'], index=np.arange(201))

r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

for i in range(201):
    arr = phi[:,:,i]
    values = arr.flatten()
    
    # Interpolate the data to the Cartesian grid
    grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')
    
    # Plot and save image as PNG and JPG, reopen JPG as np array
    plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
    plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
    plt.axis("off")
    plt.savefig('analysis2.2.png', bbox_inches='tight', pad_inches=0)
    plt.savefig('analysis2.2.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.2.jpg')
    im_array = np.asarray(im)

    # Create convert function to use to convert im_array (203x203) back to raw values
    phimax, phimin = np.max(grid_z), np.min(grid_z)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through im_array and replace RGB by raw in copy (203x203)
    copy = grid_z.copy()
    for row_index, row in enumerate(im_array):
        for col_index, col in enumerate(row):
            copy[row_index][col_index] = convertRGBToRaw(col[0])

    # Get output_array from copy by computing nearest pixels
    output_array = np.zeros((101, 227))
    for rad in range(101):
        for ang in range(227):
            # Polar coordinates
            radius = r[rad]
            angle = theta[ang]
            
            # Convert to Cartesian coordinates
            x_cart = radius * np.cos(angle)
            y_cart = radius * np.sin(angle)
            
            # Find the nearest pixel in the Cartesian grid
            x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
            y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
            
            # Get the raw values from copy and put into output array
            output_array[rad, ang] = copy[y_nearest, x_nearest]
            
    # Compute compression ratio and RMSE's, add to dataframe
    in_imsize = os.stat('analysis2.2.png').st_size
    out_imsize = os.stat('analysis2.2.jpg').st_size
    df.at[i, 'CR2.2'] = in_imsize/out_imsize
    
    diff_array = output_array - arr
    rmse = np.sqrt(np.mean(diff_array**2))
    df.at[i, 'RMSE2.2'] = rmse
df.to_pickle("analysis2.2.pkl")

In [26]:
# Testing single images
i = 201

phi = np.load("Phi_201snap.npy")
r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

arr = phi[:,:,i]
values = arr.flatten()
    
# Interpolate the data to the Cartesian grid
grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')
    
# Plot and save image
plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
plt.axis("off")
plt.savefig('analysis2.2.jpg', bbox_inches='tight', pad_inches=0)
im = Image.open('analysis2.2.jpg')
im_array = np.asarray(im)

# Create convert function to use to convert im_array back to raw values
phimax, phimin = np.max(grid_z), np.min(grid_z)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin

# Loop through im_array and replace RGB by raw in copy
copy = grid_z.copy()
for i, row in enumerate(im_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])

output_array = np.zeros((101, 227))
for rad in range(101):
    for ang in range(227):
        # Polar coordinates
        radius = r[rad]
        angle = theta[ang]
            
        # Convert to Cartesian coordinates
        x_cart = radius * np.cos(angle)
        y_cart = radius * np.sin(angle)
            
        # Find the nearest pixel in the Cartesian grid
        x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
        y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
            
        # Get the raw values from copy and put into output array
        output_array[rad, ang] = copy[y_nearest, x_nearest]
        
# Compute norm, add to dataframe
diff_array = output_array - arr
norm = np.linalg.norm(diff_array)
norm

IndexError: index 201 is out of bounds for axis 2 with size 201